<a href="https://colab.research.google.com/github/Kimjibeom/LSTM/blob/main/LSTM(%EC%88%98%EC%A0%95%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split 
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
import os
import json
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from pymongo import MongoClient
from dateutil import parser
from datetime import datetime, timedelta
# import lstm, time #도움을 주는 라이브러리들

In [2]:
#DB 연동
db = conn.get_database("traffic_Sejong")

In [ ]:
def make_coll(category):
    collections = db.get_collection(category)
    return collections

def upsert_DB(info, category):
    coll_category = make_coll(category)
    for i in info:
        coll_category.replace_one({"Date":i["Date"]}, i , upsert=True)

def get_DB(dateStr, category):
    coll_category = make_coll(category)
    dateISO = parser.parse(dateStr)
    obj_train = coll_category.aggregate([{"$match":{"Date":{"$lt":dateISO}}},
                                {"$project":{"_id":0, "Date":1, "High":1, "Low":1, "Open":1, "Close":1}},
                                {"$sort":{"Date":1}}])

    obj_test = coll_category.aggregate([{"$match":{"Date":{"$gte":dateISO}}},
                                {"$project":{"_id":0, "Date":1, "High":1, "Low":1, "Open":1, "Close":1}},
                                {"$sort":{"Date":1}}])
    return (list(obj_train), list(obj_test))

In [3]:
coll_list = db.list_collection_names()
print(coll_list)

['부강터널 ~ 문곡교차로', '새내4길시점 ~ 마음그린한의원', '- ~ 송악교', '여울초교앞 ~ 교육청앞', '조치원지하차도앞 ~ 조치원교', 'e마트남측교차로 ~ -', '조치원로1 ~ 상리사거리', '시청입구 ~ 교육청후문', '한누리대로고가2북측 ~ 양지초교사거리', '마음그린한의원 ~ 새내4길시점', '연동중학교입구 ~ 응암삼거리', '한누리대로지하4북측 ~ 도시통합정보센터앞사거리', '문곡교차로 ~ 부강터널', '노드_070 ~ 노드_069', '교육청앞 ~ 여울초교앞', '송학사거리 ~ 송문리', '조치원여자중학교 ~ 조치원읍사무소', '한누리대로고가2남측 ~ 노드_146', '시목교(북측) ~ -', '노드_069 ~ 노드_070', '올망초등학교 ~ 다정동', '수산IC교 북서 ~ 모개고가차도', '4-1생활권(한누리대로) ~ 반곡동(한누리대로)', '한누리대로지하5북측 ~ 한누리대로지하4남측', '양지초교사거리 ~ 한누리대로고가2북측', '- ~ 시목교(북측)', '반곡동(한누리대로) ~ 4-1생활권(한누리대로)', '한누리대로지하4남측 ~ 한누리대로지하5북측', '노드_143 ~ 노드_142', '와룡로입구 ~ 세종-공주경계', '조치원읍사무소 ~ 조치원여자중학교', '노드_142 ~ 노드_143', '노드_146 ~ 한누리대로고가2남측', '송문리 ~ 송학사거리', '도시통합정보센터앞사거리 ~ 한누리대로지하4북측', '세종-공주경계 ~ 와룡로입구', '상리사거리 ~ 조치원로1', '연서치안센터 ~ 월암교남측', '원수산로교차로3 ~ 절재로교차로', '- ~ e마트남측교차로', '다정동 ~ 올망초등학교', '갈매로교차로1 ~ 금사교고가차도동편', '월암교남측 ~ 연서치안센터', '응암삼거리 ~ 연동중학교입구', '송암교 ~ -', '문주1교북측 ~ 예양교차로남측', '부강교차로1 ~ 부강교차로2', '장남교차로 ~ 금남교앞', '송악교 ~ -', '모개고가차도 ~ 수산IC교 북서', '조치원교 ~ 조치원지하차도앞', '금사교고가차도동편 

In [18]:
a=[]   # 갈매로
b=[]   # 당산로
c=[]   # 산학리길
d=[]   # 새내로
e=[]   # 시목부강로
f=[]   # 시청대로
g=[]   # 연청로
h=[]   # 용연로
u=[]   # 일반국도36호선
k=[]   # 절재로
l=[]   # 정안세종로
m=[]   # 조치원로
n=[]   # 지방도604호선
o=[]   # 지방도96호선
p=[]   # 청연로
q=[]   # 충현로
r=[]   # 평기길
s=[]    # 한누리대로

In [19]:
for i in coll_list:
    tmp_col = db.get_collection(i)
    for j in tmp_col.find().limit(100):
        if j['도로명'] == '갈매로':
          a.append(j)
        if j['도로명'] == '당산로':
          b.append(j)
        if j['도로명'] == '산학리길':
          c.append(j)
        if j['도로명'] == '새내로':
          d.append(j)
        if j['도로명'] == '시목부강로':
          e.append(j)
        if j['도로명'] == '시청대로':
          f.append(j)
        if j['도로명'] == '연청로':
          g.append(j)                    
        if j['도로명'] == '용연로':
          h.append(j)
        if j['도로명'] == '일반국도36호선':
          u.append(j)
        if j['도로명'] == '절재로':
          k.append(j)
        if j['도로명'] == '정안세종로':
          l.append(j)
        if j['도로명'] == '조치원로':
          m.append(j)
        if j['도로명'] == '지방도604호선':
          n.append(j)
        if j['도로명'] == '지방도96호선':
          o.append(j)
        if j['도로명'] == '청연로':
          p.append(j)
        if j['도로명'] == '충현로':
          q.append(j)
        if j['도로명'] == '평기길':
          r.append(j)
        if j['도로명'] == '한누리대로':
          s.append(j)

In [20]:
print(a)

[{'_id': ObjectId('6167e9dc968ab64b238d194b'), '': '48', 'Datetime': '2019001001  12:00:00 AM', '도로명': '갈매로', '위치': '제3주차장(동측)', '지점': '노드_070 ~ 노드_069', '방향': '상행', '도로총교통량': '56', '평균통행속도': '52.125', '점유율': '1.1375', '휴일': '1'}, {'_id': ObjectId('6167e9dc968ab64b238d194c'), '': '49', 'Datetime': '2019001001  1:00:00 AM', '도로명': '갈매로', '위치': '제3주차장(동측)', '지점': '노드_070 ~ 노드_069', '방향': '상행', '도로총교통량': '78', '평균통행속도': '56.5', '점유율': '1.25375', '휴일': '1'}, {'_id': ObjectId('6167e9dc968ab64b238d194d'), '': '50', 'Datetime': '2019001001  2:00:00 AM', '도로명': '갈매로', '위치': '제3주차장(동측)', '지점': '노드_070 ~ 노드_069', '방향': '상행', '도로총교통량': '45', '평균통행속도': '58.875', '점유율': '0.91375', '휴일': '1'}, {'_id': ObjectId('6167e9dc968ab64b238d194e'), '': '51', 'Datetime': '2019001001  3:00:00 AM', '도로명': '갈매로', '위치': '제3주차장(동측)', '지점': '노드_070 ~ 노드_069', '방향': '상행', '도로총교통량': '16', '평균통행속도': '57', '점유율': '0.955', '휴일': '1'}, {'_id': ObjectId('6167e9dc968ab64b238d194f'), '': '52', 'Datetime': '2019001001  4:00:0

In [24]:
target = []
for x in a:
  if x['지점'].find('갈매로교차로1 ~ 금사교고가차도동편') !=-1:
    target.append(x)
print(target)

[{'_id': ObjectId('6167e9ab968ab64b238bbbdb'), '': '0', 'Datetime': '2019001001  12:00:00 AM', '도로명': '갈매로', '위치': '기쁨뜰공원(정안)', '지점': '갈매로교차로1 ~ 금사교고가차도동편', '방향': '상행', '도로총교통량': '28', '평균통행속도': '57.2857', '점유율': '0.817143', '휴일': '1'}, {'_id': ObjectId('6167e9ab968ab64b238bbbdc'), '': '1', 'Datetime': '2019001001  1:00:00 AM', '도로명': '갈매로', '위치': '기쁨뜰공원(정안)', '지점': '갈매로교차로1 ~ 금사교고가차도동편', '방향': '상행', '도로총교통량': '8', '평균통행속도': '53.6', '점유율': '0.79', '휴일': '1'}, {'_id': ObjectId('6167e9ab968ab64b238bbbdd'), '': '2', 'Datetime': '2019001001  2:00:00 AM', '도로명': '갈매로', '위치': '기쁨뜰공원(정안)', '지점': '갈매로교차로1 ~ 금사교고가차도동편', '방향': '상행', '도로총교통량': '6', '평균통행속도': '58.75', '점유율': '0.6575', '휴일': '1'}, {'_id': ObjectId('6167e9ab968ab64b238bbbde'), '': '3', 'Datetime': '2019001001  3:00:00 AM', '도로명': '갈매로', '위치': '기쁨뜰공원(정안)', '지점': '갈매로교차로1 ~ 금사교고가차도동편', '방향': '상행', '도로총교통량': '6', '평균통행속도': '63.6', '점유율': '0.792', '휴일': '1'}, {'_id': ObjectId('6167e9ab968ab64b238bbbdf'), '': '4', 'Datetime': '2019001001

In [ ]:
'''
for i in coll_list:
    if not i.endswith('predicts'):
        print(i)
'''

조치원지하차도앞 ~ 조치원교
송학사거리 ~ 송문리
노드_069 ~ 노드_070
조치원교 ~ 조치원지하차도앞
송암교 ~ -
연동중학교입구 ~ 응암삼거리
문곡교차로 ~ 부강터널
한누리대로고가2남측 ~ 노드_146
세종-공주경계 ~ 와룡로입구
와룡로입구 ~ 세종-공주경계
모개고가차도 ~ 수산IC교 북서
user_lists
- ~ 시목교(북측)
수산IC교 북서 ~ 모개고가차도
올망초등학교 ~ 다정동
한누리대로지하5북측 ~ 한누리대로지하4남측
노드_142 ~ 노드_143
상리사거리 ~ 조치원로1
시청입구 ~ 교육청후문
- ~ e마트남측교차로
조치원로1 ~ 상리사거리
e마트남측교차로 ~ -
교육청앞 ~ 여울초교앞
부강교차로1 ~ 부강교차로2
노드_070 ~ 노드_069
- ~ 송악교
다정동 ~ 올망초등학교
문주1교북측 ~ 예양교차로남측
장남교차로 ~ 금남교앞
조치원읍사무소 ~ 조치원여자중학교
응암삼거리 ~ 연동중학교입구
여울초교앞 ~ 교육청앞
노드_146 ~ 한누리대로고가2남측
송악교 ~ -
반곡동(한누리대로) ~ 4-1생활권(한누리대로)
송문리 ~ 송학사거리
원수산로교차로3 ~ 절재로교차로
조치원여자중학교 ~ 조치원읍사무소
한누리대로지하4남측 ~ 한누리대로지하5북측
연서치안센터 ~ 월암교남측
도시통합정보센터앞사거리 ~ 한누리대로지하4북측
한누리대로고가2북측 ~ 양지초교사거리
금사교고가차도동편 ~ 갈매로교차로1
4-1생활권(한누리대로) ~ 반곡동(한누리대로)
마음그린한의원 ~ 새내4길시점
월암교남측 ~ 연서치안센터
갈매로교차로1 ~ 금사교고가차도동편
한누리대로지하4북측 ~ 도시통합정보센터앞사거리
부강터널 ~ 문곡교차로
노드_143 ~ 노드_142
양지초교사거리 ~ 한누리대로고가2북측
시목교(북측) ~ -
새내4길시점 ~ 마음그린한의원


In [ ]:
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["10.244.1.73:8899",
                   "10.244.3.61:8899"]
    },
    'task': {'type': 'worker', 'index': 0}
})
multiworker_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()


data = pd.read_csv('a.csv')

Instructions for updating:
use distribute.MultiWorkerMirroredStrategy instead
INFO:tensorflow:Enabled multi-worker collective ops with available devices: ['/job:worker/replica:0/task:0/device:CPU:0']
INFO:tensorflow:Waiting for the cluster, timeout = inf


KeyboardInterrupt: ignored

In [ ]:
# Normalization - 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scale_clos = ['도로총교통량','평균통행속도','점유율']
df_scaled = scaler.fit_transform(data[scale_clos])

df_scaled = pd.DataFrame(data=df_scaled,columns=scale_clos)

train = df_scaled[:19000] # 학습 데이터
test = df_scaled[19000:] # 테스트 데이터

In [ ]:
# 학습 시킬 데이터 셋 생성 함수
# 30일 기반으로 데이터 셋 묶기
def make_dataset(data,label,window_size=30):
  feature_list =[] # 학습
  label_list =[] # 답
  for i in range(len(data)-window_size):
    feature_list.append(np.array(data.iloc[i:i+window_size]))
    label_list.append(np.array(label.iloc[i+window_size]))
  return np.array(feature_list),np.array(label_list)
feature_cols = ['도로총교통량','평균통행속도'] # 학습 컬럼
label_cols = ['점유율'] # 답 컬럼

train_feature = train[feature_cols]
train_label = train[label_cols]

# train dataset
train_feature,train_label = make_dataset(train_feature,train_label,30)

# train,test set 생성
X_train,X_test,y_train,y_test = \
train_test_split(train_feature,train_label,test_size=0.8,shuffle=False)

test_feature = test[feature_cols]
test_label = test[label_cols]

# test dataset(실제 예측 데이터)
test_feature,test_label = make_dataset(test_feature,test_label,30)

In [ ]:
with multiworker_strategy.scope():
    model = models.Sequential()
    model.add(layers.LSTM(
        32            # 노드 개수
        ,input_shape=(train_feature.shape[1],train_feature.shape[2])
        # input_dim 입력 데이터의 피쳐 개수
        ,activation='relu' # 모델을 활성화 함수 선형-> 비선형
        ,return_sequences = False # 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴 False => 하나의 은닉 상태값 출력
    ))

    model.add(layers.Dense(1)) # 예측하고자하는 target이 하나 (output)
    
    model.compile(loss='mean_squared_error',optimizer='adam')
callbacks = [ModelCheckpoint(filepath='./chk_point/a.chk', save_freq='epoch')]
model.fit(X_train,y_train,epochs=100,batch_size=32
               ,validation_data=(X_test,y_test)
               ,callbacks=callbacks
               ,shuffle = False
      )